## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Jieshi,CN,22.8134,115.8257,76.57,84,18,16.62,few clouds
1,1,Tasiilaq,GL,65.6145,-37.6368,41.16,91,98,5.12,overcast clouds
2,2,Albany,US,42.6001,-73.9662,90.25,51,45,1.99,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,76.64,84,98,15.88,overcast clouds
4,4,Namibe,AO,-15.1961,12.1522,74.07,54,50,8.39,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Jieshi,CN,22.8134,115.8257,76.57,84,18,16.62,few clouds
3,3,Rikitea,PF,-23.1203,-134.9692,76.64,84,98,15.88,overcast clouds
7,7,Kapaa,US,22.0752,-159.3190,82.72,71,40,5.01,scattered clouds
8,8,Mizdah,LY,31.4451,12.9801,78.93,21,10,11.56,clear sky
14,14,Puerto Ayora,EC,-0.7393,-90.3518,80.56,89,35,5.99,scattered clouds
20,20,Massillon,US,40.7967,-81.5215,87.37,56,40,1.01,scattered clouds
21,21,Atuona,PF,-9.8000,-139.0333,79.93,64,63,6.69,broken clouds
22,22,Cozumel,MX,20.5083,-86.9458,87.87,62,75,9.22,broken clouds
25,25,Bom Jesus,BR,-9.0744,-44.3586,80.53,35,100,5.91,overcast clouds
42,42,Dong Hoi,VN,17.4833,106.6000,82.38,74,100,6.67,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        223
City           223
Country        221
Lat            223
Lng            223
Max Temp       223
Humidity       223
Cloudiness     223
Wind Speed     223
Description    223
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Jieshi,CN,22.8134,115.8257,76.57,84,18,16.62,few clouds
3,3,Rikitea,PF,-23.1203,-134.9692,76.64,84,98,15.88,overcast clouds
7,7,Kapaa,US,22.0752,-159.3190,82.72,71,40,5.01,scattered clouds
8,8,Mizdah,LY,31.4451,12.9801,78.93,21,10,11.56,clear sky
14,14,Puerto Ayora,EC,-0.7393,-90.3518,80.56,89,35,5.99,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
666,666,Biltine,TD,14.5333,20.9167,89.65,17,80,3.74,broken clouds
670,670,Olinda,BR,-8.0089,-34.8553,78.53,69,40,11.56,scattered clouds
679,679,Gizo,SB,-8.1030,156.8419,82.29,77,26,5.91,scattered clouds
681,681,Matagami,CA,49.7502,-77.6328,84.31,39,1,9.46,clear sky


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Jieshi,CN,76.57,few clouds,22.8134,115.8257,
3,Rikitea,PF,76.64,overcast clouds,-23.1203,-134.9692,
7,Kapaa,US,82.72,scattered clouds,22.0752,-159.3190,
8,Mizdah,LY,78.93,clear sky,31.4451,12.9801,
14,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,
20,Massillon,US,87.37,scattered clouds,40.7967,-81.5215,
21,Atuona,PF,79.93,broken clouds,-9.8000,-139.0333,
22,Cozumel,MX,87.87,broken clouds,20.5083,-86.9458,
25,Bom Jesus,BR,80.53,overcast clouds,-9.0744,-44.3586,
42,Dong Hoi,VN,82.38,overcast clouds,17.4833,106.6000,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv" 
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))